# **QUESTION 1**

In [ ]:
################################### Question 1 (Preparation and Data Import) ###################################

import pandas as pd
import numpy as np
import statsmodels.api as sm
from numpy.linalg import inv
from scipy.optimize import minimize
from scipy.stats import chi2
from scipy.stats import norm
import pyblp


np.random.seed(42)

df1 = pd.read_csv("Sales_Hausman.csv",sep=",")
df2 = pd.read_csv("OTC_Demographics.csv",sep="\t")
df3 = pd.read_csv("OTC_Instruments.csv",sep="\t")


df1.head()


In [ ]:
################################### Summary statistics for the table 1 ###################################

# Calculate the total sales
total_sales = df1['sales_'].sum()

# Calculate the sales share for each brand
brand_sales = df1.groupby('brand')['sales_'].sum()


# Generate share variables for each brand
share_1 = brand_sales[1] / total_sales
share_2 = brand_sales[2] / total_sales
share_3 = brand_sales[3] / total_sales
share_4 = brand_sales[4] / total_sales
share_5 = brand_sales[5] / total_sales
share_6 = brand_sales[6] / total_sales
share_7 = brand_sales[7] / total_sales
share_8 = brand_sales[8] / total_sales
share_9 = brand_sales[9] / total_sales
share_10 = brand_sales[10] / total_sales
share_11 = brand_sales[11] / total_sales

# Print the share variables
print(f"share_1: {share_1}")
print(f"share_2: {share_2}")
print(f"share_3: {share_3}")
print(f"share_4: {share_4}")
print(f"share_5: {share_5}")
print(f"share_6: {share_6}")
print(f"share_7: {share_7}")
print(f"share_8: {share_8}")
print(f"share_9: {share_9}")
print(f"share_10: {share_10}")
print(f"share_11: {share_11}")


In [ ]:
# Generate dummy variables for adv, tyl, bay, and sto
df1['adv'] = df1['brand'].isin([4, 5, 6]).astype(int)
df1['tyl'] = df1['brand'].isin([1, 2, 3]).astype(int)
df1['bay'] = df1['brand'].isin([7, 8, 9]).astype(int)
df1['sto'] = df1['brand'].isin([10, 11]).astype(int)

# Display the first few rows to verify
df1.head()

In [ ]:
######################################### Weighted average price and cost for each brand #########################################

for i in range(1, 12):
    weighted_avg_price_i = (df1[df1['brand'] == i]['sales_'] * df1[df1['brand'] == i]['price_']).sum() / df1[df1['brand'] == i]['sales_'].sum()
    print(f"weighted_avg_price_{i}: {weighted_avg_price_i}")

# Calculate the weighted average cost (wholesale price) for each brand

for i in range(1, 12):
    weighted_avg_cost_i = (df1[df1['brand'] == i]['sales_'] * df1[df1['brand'] == i]['cost_']).sum() / df1[df1['brand'] == i]['sales_'].sum()
    print(f"weighted_avg_cost_{i}: {weighted_avg_cost_i}")


# **QUESTION 2**

In [ ]:
####################################### Question 2 (Data Preparation) #######################################

df1['sales_per_count'] = df1['sales_'] / df1['count']

df1['share_0'] = 1 - df1.groupby(['store', 'week'])['sales_per_count'].transform('sum')

df1['Y'] = np.log(df1['sales_per_count']) - np.log(df1['share_0'])

df1['market_id'] = df1['store'].astype(str) + '_' + df1['week'].astype(str)
df1.head()

df1.head()

# **QUESTION 2a**

In [ ]:
##################################### OLS Q2.(a) #####################################

# Define the independent variables
X = df1[['prom_', 'price_']]

# Add a constant to the independent variables
X = sm.add_constant(X)

# Define the dependent variable
Y = df1['Y']

# Fit the regression model
model = sm.OLS(Y, X).fit()

# Print the regression results
print(model.summary())

# **QUESTION 2b**

In [ ]:
######################################## Generate Dummies for Product Type ########################################

# Create dummy variables
df1['tyl'] = df1['brand'].isin([1, 2, 3]).astype(int)
df1['adv'] = df1['brand'].isin([4, 5, 6]).astype(int)
df1['bay'] = df1['brand'].isin([7, 8, 9]).astype(int)
df1['sto'] = df1['brand'].isin([10, 11]).astype(int)

# Display the first few rows to verify
df1.head()

# Create separate dummy variables for each brand
for brand in df1['brand'].unique():
    df1[f'brand_{brand}'] = (df1['brand'] == brand).astype(int)

# Display the first few rows to verify
df1.head()

In [ ]:
##################################### OLS Q2.(b) #####################################

# Define the independent variables including the dummy variables
X = df1[['price_', 'prom_', 'brand_1', 'brand_2', 'brand_3', 'brand_4', 'brand_5', 'brand_6', 'brand_7', 'brand_8', 'brand_9', 'brand_10', 'brand_11']] 


# Add a constant to the independent variables
X = sm.add_constant(X)

# Define the dependent variable
Y = df1['Y']

# Fit the regression model
model_with_dummies = sm.OLS(Y, X).fit()

# Print the regression results
print(model_with_dummies.summary())

# **QUESTION 2c**

In [ ]:
############################################### Fixed Effects for Everything ###############################################

# Generate new variables for each unique combination of store number and tyl, adv, bay, and sto
for store in df1['store'].unique():
    df1[f'tyl_store_{store}'] = df1['tyl'] * (df1['store'] == store).astype(int)
    df1[f'adv_store_{store}'] = df1['adv'] * (df1['store'] == store).astype(int)
    df1[f'bay_store_{store}'] = df1['bay'] * (df1['store'] == store).astype(int)
    df1[f'sto_store_{store}'] = df1['sto'] * (df1['store'] == store).astype(int)

# Display the first few rows to verify
df1.head()


In [ ]:
##################################### OLS Q2.(c) Fixed Effects Model #####################################

# Define the independent variables including the dummy variables
X_fixed_effects = df1[['price_', 'prom_'] + [col for col in df1.columns if '_store' in col]]


# Add a constant to the independent variables
X_fixed_effects = sm.add_constant(X_fixed_effects)

# Define the dependent variable
Y = df1['Y']

# Fit the regression model
fixed_effects_model = sm.OLS(Y, X_fixed_effects).fit()


# Print the regression results
print(fixed_effects_model.summary())

# **QUESTION 2d**

In [ ]:
####################################### OLS Q2.(d) #####################################

# Define the independent variables and the instrument
X = df1[['prom_']]
X = sm.add_constant(X)

instrument = df1['cost_']

# First stage: regress price_ on the instrument and other exogenous variables
first_stage = sm.OLS(df1['price_'], sm.add_constant(instrument)).fit()
df1['price_hat'] = first_stage.fittedvalues

# Second stage: regress Y on the predicted values of price_ and other exogenous variables
X['price_hat'] = df1['price_hat']
second_stage = sm.OLS(Y, X).fit()

# Print the regression results
print(second_stage.summary())



In [ ]:
##################################### IV with Dummies #####################################

# Define the independent variables and the instrument
X = df1[['prom_', 'brand_1', 'brand_2', 'brand_3', 'brand_4', 'brand_5', 'brand_6', 'brand_7', 'brand_8', 'brand_9', 'brand_10', 'brand_11']]
X = sm.add_constant(X)

instrument = df1['cost_']

# First stage: regress price_ on the instrument and other exogenous variables
first_stage = sm.OLS(df1['price_'], sm.add_constant(instrument)).fit()
df1['price_hat'] = first_stage.predict()

# Second stage: regress Y on the predicted values of price_ and other exogenous variables
X['price_hat'] = df1['price_hat']
second_stage = sm.OLS(Y, X).fit()

# Print the regression results
print(second_stage.summary())



In [ ]:
##################################### IV with Fixed Effects #####################################

# Define the independent variables and the instrument
X_fixed_effects = df1[['prom_'] + [col for col in df1.columns if '_store' in col]]
X_fixed_effects = sm.add_constant(X_fixed_effects)
instrument = df1['cost_']

# First stage: regress price_ on the instrument and other exogenous variables
first_stage = sm.OLS(df1['price_'], sm.add_constant(instrument)).fit()
df1['price_hat'] = first_stage.fittedvalues

# Second stage: regress Y on the predicted values of price_ and other exogenous variables
X_fixed_effects['price_hat'] = df1['price_hat']
model_iv_fixed_effects = sm.OLS(Y, X_fixed_effects).fit()

# Print the regression results
print(model_iv_fixed_effects.summary())


#print t statistic (-57.541554716524786) 
print(model_iv_fixed_effects.tvalues['price_hat'])
#print confidence interval (-0.158776 -0.148316) for "price_" coefficient
print(model_iv_fixed_effects.conf_int(alpha=0.05, cols=None))

# Print the coefficient for price
print(f"Price coefficient: {model_iv_fixed_effects.params['price_hat']}")



# **QUESTION 2e**

In [ ]:
################################################# Code used to generate Hausman Variable #################################################
### In order to save time, I added this column to the initial data set and saved it as a new CSV file. ###
# Define a function to calculate the Hausman variable
#def calculate_hausman(row):
    #other_stores = df1[(df1['store'] != row['store']) & (df1['week'] == row['week']) & (df1['brand'] == row['brand'])]
    #if not other_stores.empty:
    #    weighted_avg_price_other_stores = (other_stores['sales_'] * other_stores['price_']).sum() / other_stores['sales_'].sum()
    #else:
    #   weighted_avg_price_other_stores = np.nan
    #return weighted_avg_price_other_stores

# Apply the function to each row in the dataframe
#['Hausman'] = df1.apply(calculate_hausman, axis=1)

# Display the first few rows to verify
#df1.head()

# Add the Hausman variable to the original dataframe

#dfHausman = pd.read_csv("OTC_Sales.csv",sep="\t")
#dfHausman['Hausman'] = df1['Hausman']

# Save the updated dataframe to a new CSV file
#dfHausman.to_csv("Sales_Hausman.csv", index=False)

In [ ]:
# First stage: regress price_ on the instrument Hausman
first_stage_hausman = sm.OLS(df1['price_'], sm.add_constant(df1['Hausman'])).fit()
df1['price_hat_hausman'] = first_stage_hausman.fittedvalues

# Second stage: regress Y on the predicted values of price_ and prom_
X_hausman = df1[['prom_']]
X_hausman = sm.add_constant(X_hausman)
X_hausman['price_hat_hausman'] = df1['price_hat_hausman']
second_stage_hausman = sm.OLS(Y, X_hausman).fit()

# Print the regression results
print(second_stage_hausman.summary())

In [ ]:
# Define the independent variables including the dummy variables
X_dummies_hausman = df1[['prom_', 'price_hat_hausman', 'brand_1', 'brand_2', 'brand_3', 'brand_4', 'brand_5', 'brand_6', 'brand_7', 'brand_8', 'brand_9', 'brand_10', 'brand_11']]

# Add a constant to the independent variables
X_dummies_hausman = sm.add_constant(X_dummies_hausman)

# Define the dependent variable
Y = df1['Y']

# Fit the regression model
second_stage_dummies_hausman = sm.OLS(Y, X_dummies_hausman).fit()

# Print the regression results
print(second_stage_dummies_hausman.summary())

In [ ]:
# Define the independent variables and the instrument
X_fixed_effects_hausman = df1[['prom_'] + [col for col in df1.columns if '_store' in col]]
X_fixed_effects_hausman = sm.add_constant(X_fixed_effects_hausman)
instrument_hausman = df1['Hausman']

# First stage: regress price_ on the instrument Hausman and other exogenous variables
first_stage_hausman = sm.OLS(df1['price_'], sm.add_constant(instrument_hausman)).fit()
df1['price_hat_hausman'] = first_stage_hausman.fittedvalues

# Second stage: regress Y on the predicted values of price_ and other exogenous variables
X_fixed_effects_hausman['price_hat_hausman'] = df1['price_hat_hausman']
model_iv_fixed_effects_hausman = sm.OLS(Y, X_fixed_effects_hausman).fit()

# Print the regression results
print(model_iv_fixed_effects_hausman.summary())

# Print the coefficient for price
print(f"Price coefficient: {model_iv_fixed_effects_hausman.params['price_hat_hausman']}")

# **QUESTION 2g**

In [ ]:
# Define the coefficient of price from model (a)
price_coefficient = -0.0514

# Calculate the elasticity
df1['elasticity'] = price_coefficient * df1['price_'] * (1 - df1['sales_per_count'])

# Display the first few rows to verify
df1.head()

# Now calculate the weighted average of elasticity per product (tyl, adv, bay, sto)
elasticity_tyl = (df1['sales_'] * df1['elasticity'] * df1['tyl']).sum() / df1['sales_'].sum()
elasticity_adv = (df1['sales_'] * df1['elasticity'] * df1['adv']).sum() / df1['sales_'].sum()
elasticity_bay = (df1['sales_'] * df1['elasticity'] * df1['bay']).sum() / df1['sales_'].sum()
elasticity_sto = (df1['sales_'] * df1['elasticity'] * df1['sto']).sum() / df1['sales_'].sum()

# Print the elasticity for each product
print(f"Elasticity for tyl: {elasticity_tyl}")
print(f"Elasticity for adv: {elasticity_adv}")
print(f"Elasticity for bay: {elasticity_bay}")
print(f"Elasticity for sto: {elasticity_sto}")


In [ ]:
# Define the coefficient of price from model (b)
price_coefficient_b = model_with_dummies.params['price_']
print(price_coefficient_b)
# Calculate the elasticity
df1['elasticity_b'] = price_coefficient_b * df1['price_'] * (1 - df1['sales_per_count'])

# Display the first few rows to verify
df1.head()

# Now calculate the weighted average of elasticity per product (tyl, adv, bay, sto)
elasticity_tyl_b = (df1['sales_'] * df1['elasticity_b'] * df1['tyl']).sum() / df1['sales_'].sum()
elasticity_adv_b = (df1['sales_'] * df1['elasticity_b'] * df1['adv']).sum() / df1['sales_'].sum()
elasticity_bay_b = (df1['sales_'] * df1['elasticity_b'] * df1['bay']).sum() / df1['sales_'].sum()
elasticity_sto_b = (df1['sales_'] * df1['elasticity_b'] * df1['sto']).sum() / df1['sales_'].sum()

# Print the elasticity for each product
print(f"Elasticity for tyl (part b): {elasticity_tyl_b}")
print(f"Elasticity for adv (part b): {elasticity_adv_b}")
print(f"Elasticity for bay (part b): {elasticity_bay_b}")
print(f"Elasticity for sto (part b): {elasticity_sto_b}")


In [ ]:
# Define the coefficient of price from model (c)
price_coefficient_c = fixed_effects_model.params['price_']

print(price_coefficient_c)

# Calculate the elasticity
df1['elasticity_c'] = price_coefficient_c * df1['price_'] * (1 - df1['sales_per_count'])

# Display the first few rows to verify
df1.head()

# Now calculate the weighted average of elasticity per product (tyl, adv, bay, sto)
elasticity_tyl_c = (df1['sales_'] * df1['elasticity_c'] * df1['tyl']).sum() / df1['sales_'].sum()
elasticity_adv_c = (df1['sales_'] * df1['elasticity_c'] * df1['adv']).sum() / df1['sales_'].sum()
elasticity_bay_c = (df1['sales_'] * df1['elasticity_c'] * df1['bay']).sum() / df1['sales_'].sum()
elasticity_sto_c = (df1['sales_'] * df1['elasticity_c'] * df1['sto']).sum() / df1['sales_'].sum()

# Print the elasticity for each product
print(f"Elasticity for tyl (part c): {elasticity_tyl_c}")
print(f"Elasticity for adv (part c): {elasticity_adv_c}")
print(f"Elasticity for bay (part c): {elasticity_bay_c}")
print(f"Elasticity for sto (part c): {elasticity_sto_c}")

# **QUESTION 3**

In [ ]:
######################################### Data Preparation for Question 3 #########################################

# Generate a single dummy variable for brands 1 to 9
df1['brand_'] = df1['brand'].isin(range(1, 10)).astype(int)

df_merged = pd.merge(df1, df2, on=['store'], how='left')
df_merged = pd.merge(df_merged, df3, on=['store','week','brand'], how='left')

df_merged['cost_']=df_merged['cost__x'].fillna(0)

df_merged.drop(columns=['cost__x'], inplace=True)

# Generate a value randomly drawn from N(0,1) for each observation
df_merged['V'] = np.random.normal(0, 1, df_merged.shape[0])


# Generate PI variable which is the product of price_ and e^(income)
df_merged['PI'] = df_merged['price_'] * df_merged['income']
df_merged

In [ ]:

########################################## Contraction Mapping ###################################################
#The model that I'm considering is u=beta*prom_+alpha*price_+sigma_i*PI+beta_i*V*brand_+Zeta+epsilon of the U_i into
# two parts (excluding epsilon): delta_i=beta*prom__i+alpha*price__i+sigma_i*PI_i+Zeta_i and mu_i=sigma_b*V_i*brand__i.
# define 0_sigma_i=integral (e^(delta_i+mu_i))/(1+sum_{k}e^(delta_k+mu_k))f(v)dv where f is the density of standard normal
# and V is the vector 'V' from out dataframe. We will need numerical integration of this in the end.  Now the steps are, 
# fix sigma_b at some level and start the procedure: pick 0_delta which is the vector of 0_delta_i's where 0_delta_i for 
# a given individual is its corresponding value of Y variable from the data frame. Then update 0_delta vector to 1_delta 
# vector the following way-> 1_delta_i=0_delta_i+log(sales_per_count_i)-log(0_sigma_i). 
# continue this process until the distance between the VECTORS t_delta and {t+1}_delta converges to 0 (up to 3 decimal points) 




In [ ]:


# Calculate the initial delta
delta_0 = df_merged['Y']
df_merged['delta_0'] = delta_0

# Generate mu for each entry
sigma_b = 0.00000001
df_merged['mu'] = sigma_b * df_merged['V'] * df_merged['brand_']

# Calculate sigma_0
sigma_0denom = 1 + np.exp(df_merged['delta_0'] + df_merged['mu']).sum()
sigma_0numer = np.exp(df_merged['delta_0'] + df_merged['mu'])
sigma_0 = sigma_0numer / sigma_0denom
df_merged['sigma_0'] = sigma_0

# Update delta
delta_1 = delta_0 + np.log(df_merged['sales_per_count']) - np.log(sigma_0)
df_merged['delta_1'] = delta_1

# Calculate sigma_1

sigma_1denom = 1 + np.exp(df_merged['delta_1'] + df_merged['mu']).sum()
sigma_1numer = np.exp(df_merged['delta_1'] + df_merged['mu'])
sigma_1 = sigma_1numer / sigma_1denom

df_merged['sigma_1'] = sigma_1


# update delta

delta_2 = delta_1 + np.log(df_merged['sales_per_count']) - np.log(sigma_1)
df_merged['delta_2'] = delta_2

# Calculate sigma_2

sigma_2denom = 1 + np.exp(df_merged['delta_2'] + df_merged['mu']).sum()
sigma_2numer = np.exp(df_merged['delta_2'] + df_merged['mu'])
sigma_2 = sigma_2numer / sigma_2denom

df_merged['sigma_2'] = sigma_2


# Continue iterating this way 5-6 times

for t in range(3, 8):
    # Calculate sigma_t
    sigma_t_denom = 1 + np.exp(df_merged[f'delta_{t-1}'] + df_merged['mu']).sum()
    sigma_t_numer = np.exp(df_merged[f'delta_{t-1}'] + df_merged['mu'])
    sigma_t = sigma_t_numer / sigma_t_denom
    df_merged[f'sigma_{t}'] = sigma_t

    # Update delta_t
    delta_t = df_merged[f'delta_{t-1}'] + np.log(df_merged['sales_per_count']) - np.log(sigma_t)
    df_merged[f'delta_{t}'] = delta_t

# Display the final delta and sigma values
df_merged[[f'delta_{t}' for t in range(8)] + [f'sigma_{t}' for t in range(8)]].head()





# **PYBLP**

In [ ]:
##################    GENERAL  METHOD      #############################

import pyblp
import pandas as pd

# Load the data
product_data = df_merged.copy()

product_data['market_ids'] = product_data['market_id']

product_data['prices']=product_data['price_']

product_data['shares']=product_data['sales_per_count']  

demand_instruments=product_data[['cost_','avoutprice', 'pricestore1', 'pricestore2', 'pricestore3', 'pricestore4', 'pricestore5', 'pricestore6', 'pricestore7', 'pricestore8', 'pricestore9', 'pricestore10', 'pricestore11', 'pricestore12', 'pricestore13', 'pricestore14', 'pricestore15', 'pricestore16', 'pricestore17', 'pricestore18', 'pricestore19', 'pricestore20', 'pricestore21', 'pricestore22', 'pricestore23', 'pricestore24', 'pricestore25', 'pricestore26', 'pricestore27', 'pricestore28', 'pricestore29', 'pricestore30']]


X1_formulation = pyblp.Formulation('0 + prices + PI + prom_ ', absorb='C(brand)')
X2_formulation = pyblp.Formulation('1 + brand_')
product_formulations = (X1_formulation, X2_formulation)
product_formulations

pr_integration = pyblp.Integration('product', size=5)


problem = pyblp.Problem(product_formulations, product_data,  integration=pr_integration, )
problem

logit_results = problem.solve(sigma=np.eye(2))
logit_results

In [ ]:
###################################### BLP with instruments ###########################################

import pyblp
import pandas as pd

# 1. Prepare product data
product_data = df_merged.copy()
product_data['priceincome'] = product_data['PI']
product_data['firm_ids'] = product_data['brand']   # older PyBLP requires this when using X3
product_data['market_ids'] = product_data['market_id']
product_data['shares'] = product_data['sales_per_count']

# IMPORTANT: rename price_ -> 'prices' (plural)
product_data['prices'] = product_data['price_']

# If you want a random brand dummy:
product_data['brand_dummy'] = (
    product_data[['brand_1','brand_2','brand_3','brand_4',
                  'brand_5','brand_6','brand_7','brand_8',
                  'brand_9','brand_10','brand_11']]
    .max(axis=1)
)

# 2. Define X1, X2, X3 as separate Formulations
X1_formulation = pyblp.Formulation(
    '0 + prices + prom_ + brand_1 + brand_2 + brand_3 + brand_4 + '
    'brand_5 + brand_6 + brand_7 + brand_8 + brand_9 + brand_10 + brand_11 + PI'
)

X2_formulation = pyblp.Formulation('0 + brand_')

X3_formulation = pyblp.Formulation(
    '0 + cost_ + avoutprice + '
    'pricestore1 + pricestore2 + pricestore3 + pricestore4 + '
    'pricestore5 + pricestore6 + pricestore7 + pricestore8 + '
    'pricestore9 + pricestore10 + pricestore11 + pricestore12 + '
    'pricestore13 + pricestore14 + pricestore15 + pricestore16 + '
    'pricestore17 + pricestore18 + pricestore19 + pricestore20 + '
    'pricestore21 + pricestore22 + pricestore23 + pricestore24 + '
    'pricestore25 + pricestore26 + pricestore27 + pricestore28 + '
    'pricestore29 + pricestore30'
)

# 3. Agent data, 1 row per market for integration size=1
agent_data = (
    product_data[['market_ids', 'income']]
    .drop_duplicates('market_ids')
    .copy()
)
agent_data['weights'] = 1

agent_formulation = pyblp.Formulation('0 + income')

# 4. Pass formulations as SEPARATE positional arguments
problem = pyblp.Problem(
    product_formulations=(X1_formulation, X2_formulation, X3_formulation),
    product_data=product_data,
    agent_formulation=agent_formulation,
    agent_data=agent_data,
    integration=pyblp.Integration('monte_carlo', size=1)
)

# Check that X2 is recognized
print("X2 recognized by PyBLP:\n", problem.products.X2)

bfgs = pyblp.Optimization('bfgs', {'gtol': 1e-4})

# 5. Solve
results = problem.solve(sigma=np.ones((1,1)), pi=np.ones((1,1)), beta=np.ones((14,1)), optimization=bfgs)  # don't pass rfp=True in older versions
print(results)

In [ ]:
########################################## BLP without instruments #########################################

import pyblp
import pandas as pd

# 1. Prepare product data
product_data = df_merged.copy()
product_data['priceincome'] = product_data['PI']
product_data['firm_ids'] = product_data['brand']   # older PyBLP requires this when using X3
product_data['market_ids'] = product_data['market_id']
product_data['shares'] = product_data['sales_per_count']

# IMPORTANT: rename price_ -> 'prices' (plural)
product_data['prices'] = product_data['price_']

# If you want a random brand dummy:
product_data['brand_dummy'] = (
    product_data[['brand_1','brand_2','brand_3','brand_4',
                  'brand_5','brand_6','brand_7','brand_8',
                  'brand_9','brand_10','brand_11']]
    .max(axis=1)
)

# 2. Define X1, X2, X3 as separate Formulations
X1_formulation = pyblp.Formulation(
    '0 + prices + prom_ + brand_1 + brand_2 + brand_3 + brand_4 + '
    'brand_5 + brand_6 + brand_7 + brand_8 + brand_9 + brand_10 + brand_11 + PI'
)

X2_formulation = pyblp.Formulation('0 + brand_')

X3_formulation = pyblp.Formulation(
    '0 + cost_ + avoutprice + '
    'pricestore1 + pricestore2 + pricestore3 + pricestore4 + '
    'pricestore5 + pricestore6 + pricestore7 + pricestore8 + '
    'pricestore9 + pricestore10 + pricestore11 + pricestore12 + '
    'pricestore13 + pricestore14 + pricestore15 + pricestore16 + '
    'pricestore17 + pricestore18 + pricestore19 + pricestore20 + '
    'pricestore21 + pricestore22 + pricestore23 + pricestore24 + '
    'pricestore25 + pricestore26 + pricestore27 + pricestore28 + '
    'pricestore29 + pricestore30'
)

# 3. Agent data, 1 row per market for integration size=1
agent_data = (
    product_data[['market_ids', 'income']]
    .drop_duplicates('market_ids')
    .copy()
)
agent_data['weights'] = 1

agent_formulation = pyblp.Formulation('0 + income')

# 4. Pass formulations as SEPARATE positional arguments
problem = pyblp.Problem(
    product_formulations=(X1_formulation, X2_formulation),
    product_data=product_data,
    agent_formulation=agent_formulation,
    agent_data=agent_data,
    integration=pyblp.Integration('monte_carlo', size=1)
)

# Check that X2 is recognized
print("X2 recognized by PyBLP:\n", problem.products.X2)

bfgs = pyblp.Optimization('bfgs', {'gtol': 1e-4})

# 5. Solve
results = problem.solve(sigma=np.ones((1,1)), pi=np.ones((1,1)), beta=np.ones((14,1)), optimization=bfgs)  # don't pass rfp=True in older versions
print(results)

# **QUESTION 3b & c**

In [ ]:
######################## Data Preparation ########################

df_merged['price_coef'] = df_merged['income'] * 0.296 - 3.65
df_merged.head()

summary_stats = df_merged['price_coef'].describe()
print(summary_stats)

price_coefficient_b * df1['price_'] * (1 - df1['sales_per_count'])

df_merged['own_price_elasticity'] = df_merged['price_coef'] * df_merged['price_'] * (1 - df_merged['sales_per_count'])

# Filter the data for store 9 in week 10
store_week_data = df_merged[(df_merged['store'] == 9) & (df_merged['week'] == 10)]

#elasticity of Tylenol
elasticity_tyl = (store_week_data['sales_'] * store_week_data['own_price_elasticity'] * store_week_data['tyl']).sum() / store_week_data['sales_'].sum()
print(f"Elasticity for Tylenol: {elasticity_tyl}")

#elasticity of Advil
elasticity_adv = (store_week_data['sales_'] * store_week_data['own_price_elasticity'] * store_week_data['adv']).sum() / store_week_data['sales_'].sum()
print(f"Elasticity for Advil: {elasticity_adv}")

#elasticity of Bayer
elasticity_bay = (store_week_data['sales_'] * store_week_data['own_price_elasticity'] * store_week_data['bay']).sum() / store_week_data['sales_'].sum()
print(f"Elasticity for Bayer: {elasticity_bay}")

#elasticity of Store Brand
elasticity_sto = (store_week_data['sales_'] * store_week_data['own_price_elasticity'] * store_week_data['sto']).sum() / store_week_data['sales_'].sum()
print(f"Elasticity for Store Brand: {elasticity_sto}")



In [ ]:
# Filter the data for store 9 in week 10
store_week_data = df_merged[(df_merged['store'] == 9) & (df_merged['week'] == 10)]

# Define the goods and their corresponding shares and prices
goods = ['tyl', 'adv', 'bay', 'sto']
shares = {
    'tyl': share_1,
    'adv': share_2,
    'bay': share_3,
    'sto': share_4
}
prices = {
    'tyl': store_week_data[store_week_data['tyl'] == 1]['price_'].mean(),
    'adv': store_week_data[store_week_data['adv'] == 1]['price_'].mean(),
    'bay': store_week_data[store_week_data['bay'] == 1]['price_'].mean(),
    'sto': store_week_data[store_week_data['sto'] == 1]['price_'].mean()
}

# Calculate cross-price elasticity for each good with respect to the other three goods
cross_price_elasticities = {}

for good in goods:
    cross_price_elasticities[good] = {}
    for other_good in goods:
        if good != other_good:
            cross_price_elasticities[good][other_good] = -prices[other_good] * shares[other_good] * store_week_data['price_coef'].mean()

# Print the cross-price elasticities
for good, elasticities in cross_price_elasticities.items():
    print(f"Cross-price elasticities for {good}:")
    for other_good, elasticity in elasticities.items():
        print(f"  With respect to {other_good}: {elasticity}")

In [ ]:
# Filter the data for store 9 in week 10
store_week_data = df_merged[(df_merged['store'] == 9) & (df_merged['week'] == 10)]

# Calculate the marginal cost for each brand
store_week_data['marginal_cost'] = store_week_data['price_'] + (1 / store_week_data['own_price_elasticity'])

# Display the marginal costs
store_week_data[['brand', 'price_', 'own_price_elasticity', 'marginal_cost', 'cost_']]




In [ ]:
# Calculate the simple average marginal cost for each brand
# Calculate the simple average marginal cost for each brand group
average_marginal_costs_group1 = store_week_data[store_week_data['brand'].isin([1, 2, 3])]['marginal_cost'].mean()
average_marginal_costs_group2 = store_week_data[store_week_data['brand'].isin([4, 5, 6])]['marginal_cost'].mean()
average_marginal_costs_group3 = store_week_data[store_week_data['brand'].isin([7, 8, 9])]['marginal_cost'].mean()
average_marginal_costs_group4 = store_week_data[store_week_data['brand'].isin([10, 11])]['marginal_cost'].mean()

# Calculate the simple average cost for each brand group
average_costs_group1 = store_week_data[store_week_data['brand'].isin([1, 2, 3])]['cost_'].mean()
average_costs_group2 = store_week_data[store_week_data['brand'].isin([4, 5, 6])]['cost_'].mean()
average_costs_group3 = store_week_data[store_week_data['brand'].isin([7, 8, 9])]['cost_'].mean()
average_costs_group4 = store_week_data[store_week_data['brand'].isin([10, 11])]['cost_'].mean()

# Print the results
print("Average Marginal Costs for each brand group:")
print(f"Group 1 (Brands 1, 2, 3): {average_marginal_costs_group1}")
print(f"Group 2 (Brands 4, 5, 6): {average_marginal_costs_group2}")
print(f"Group 3 (Brands 7, 8, 9): {average_marginal_costs_group3}")
print(f"Group 4 (Brands 10, 11): {average_marginal_costs_group4}")

print("\nAverage Costs for each brand group:")
print(f"Group 1 (Brands 1, 2, 3): {average_costs_group1}")
print(f"Group 2 (Brands 4, 5, 6): {average_costs_group2}")
print(f"Group 3 (Brands 7, 8, 9): {average_costs_group3}")
print(f"Group 4 (Brands 10, 11): {average_costs_group4}")


In [ ]:
# Calculate the overall average marginal cost
overall_avg_marginal_cost = np.mean(list(weighted_avg_marginal_cost.values()))

# Calculate the overall average elasticity
overall_avg_elasticity = np.mean([
    elasticity_brand1_store_week,
    elasticity_brand2_store_week,
    elasticity_brand3_store_week,
    elasticity_brand4_store_week,
    elasticity_brand5_store_week,
    elasticity_brand6_store_week,
    elasticity_brand7_store_week,
    elasticity_brand8_store_week,
    elasticity_brand9_store_week,
    elasticity_brand11_store_week,
    elasticity_sto_store_week
])

# Print the results
print(f"Overall Average Marginal Cost: {overall_avg_marginal_cost}")
print(f"Overall Average Elasticity: {overall_avg_elasticity}")

predicted_price=overall_avg_marginal_cost/(1+overall_avg_elasticity)
predicted_price

# **QUESTION 5**

In [ ]:
# Define the model parameters
alpha = -0.0514  # Price coefficient from model (a)
beta_prom = 0.3778387  # Promotion coefficient from model (a)

# Remove all Tylenol products from the dataset
df_no_tyl = df1[df1['tyl'] == 0]

# Recalculate the total sales without Tylenol
total_sales_no_tyl = df_no_tyl['sales_'].sum()

# Recalculate the sales share for each remaining brand
brand_sales_no_tyl = df_no_tyl.groupby('brand')['sales_'].sum()

# Generate new share variables for each remaining brand
shares_no_tyl = brand_sales_no_tyl / total_sales_no_tyl

# Predict the new shares using the model parameters
predicted_shares_no_tyl = {}
for brand, share in shares_no_tyl.items():
    price = df_no_tyl[df_no_tyl['brand'] == brand]['price_'].mean()
    prom = df_no_tyl[df_no_tyl['brand'] == brand]['prom_'].mean()
    denominator = sum(np.exp(alpha * df_no_tyl['price_'] + beta_prom * df_no_tyl['prom_']))
    predicted_share = np.exp(alpha * price + beta_prom * prom) / denominator
    predicted_shares_no_tyl[brand] = predicted_share


# Scale the predicted shares so that they sum up to 1
total_predicted_share = sum(predicted_shares_no_tyl.values())
scaled_predicted_shares_no_tyl = {brand: share / total_predicted_share for brand, share in predicted_shares_no_tyl.items()}

# Print the scaled predicted shares
for brand, scaled_share in scaled_predicted_shares_no_tyl.items():
        print(f"Scaled predicted share for brand {brand}: {scaled_share}")